<div align="center">
    <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Build%20AI%20agents%20and%20workflows%20with%20a%20simple%20API&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

## Task Definition: Sarcastic News Headline Generator using Brave Search Integration

### Overview

This task generates a sarcastic news headline on a user-provided topic. It utilizes web search to gather context and information about the topic, then uses this data to create a headline with a sarcastic tone. 

### Task Flow

1. **Input**: 
   - User provides a topic for the sarcastic news headline (e.g., "technology" or "politics").

2. **Web Search using Bravesearch Integration**: 
   - Conducts a search to find information about the topic, focusing on humorous or quirky content.
   - Gathers relevant context and amusing details for headline generation.

3. **Headline Creation**: 
   - Generates a sarcastic news headline based on the topic and gathered information.
   - Combines factual or common perceptions with humor and sarcasm.
   
4. **Output**: 
   - Produces a sarcastic, witty headline on the given topic.
   - Can be used for entertainment or as a creative writing prompt.

### Key Features

- Leverages web search to enhance contextual relevance of sarcastic content.
- Combines real-world information with humorous twists for engaging headlines.
- Adaptable to various topics, allowing for diverse and creative outputs.

```plaintext
+----------+     +------------+     +------------+     +-----------+
|  User    |     |   Brave    |     |   Agent    |     | Sarcastic |
|  Input   | --> |   Search   | --> | Processing | --> | Headline  |
| (Topic)  |     |            |     |            |     | Output    |
+----------+     +------------+     +------------+     +-----------+
      |                |                  |                  |
      |                |                  |                  |
      v                v                  v                  v
   "politics"        Find funny       Generate witty    "Area Man Still
     content          headline         Believes in        Democracy"

```

## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/f/dev/cookbooks/02-Sarcastic_News_Headline_Generator.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

Installing the Julep Client

In [10]:
!pip install julep -U --quiet

#### NOTE:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [11]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep Client with the API Key

In [12]:
from julep import Client

api_key = "" # Your API key here

# Create a client
client = Client(api_key=api_key, environment="dev")

## Creating an "agent"


Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [13]:
# Defining the agent
name = "Jarvis"
about = "The original AI conscious the Iron Man."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}


# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

## Defining a Task

Tasks in Julep are Github Actions style workflows that define long-running, multi-step actions. 
You can use them to conduct complex actions by defining them step-by-step. They have access to all Julep integrations.

To learn more about tasks, visit [Julep documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#task).

In [16]:
import yaml

Here is a Task which uses a agent to generate a sarcastic response to a given text using a DuckDuckGo search tool.

More on how to define a task can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md).

In [17]:
# defining the task definition
task_def = yaml.safe_load("""
name: Sarcasm Headline Generator

tools:
- name: brave_search
  type: integration
  integration:
    provider: brave
    setup:
      api_key: "YOU_API_KEY"

main:
- tool: brave_search
  arguments:
    query: "_.topic + ' funny'"

- prompt:
  - role: system
    content: >-
      write a sarcastic news headline on the topic of {{inputs[0].topic}}.
      Here's some more info on this: {{_}}
  unwrap: true

""")

Creating/Updating a task to generate a sarcastic response to a given text using a Intergation.

In [18]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates a execution worflow for the Task defined in the yaml file.

In [21]:
# creating an execution object
execution = client.executions.create(
    task_id=TASK_UUID,
    input={
        "topic": "elon musk"
    }
)

In [ ]:
# getting the execution details
execution = client.executions.get(execution.id)
#printing the output
execution.output

Retrieves and lists all the steps of a defined task that have been executed up to that point in time. Unlike streaming, this function does not continuously monitor the execution; it only provides a snapshot of the steps completed so far without displaying real-time updates as the task progresses.

In [ ]:
client.executions.transitions.list(execution_id=execution.id).items

Continuously monitor and stream the steps of a defined task. It retrieves and displays the transitions or execution steps of the task identified by execution.id in real-time, showing each step sequentially until the task is either completed or an error causes it to terminate.

In [ ]:
client.executions.transitions.stream(execution_id=execution.id)